# **Практика: линейная регрессия и метод наименьших квадратов**

Сразу импортируем необходимые библиотеки для работы с данными:

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline

У Василия, основателя компании «Газ-Таз-Ваз-Нефть», дела идут в гору: в этом году он открывает 100 новых скважин по добыче газа. Однако в целях оптимизации расходов и для потенциального повышения дохода Василию необходимо оценить, сколько денег будет приносить ему каждая из скважин, а также понять, какие факторы потенциально сильнейшим образом влияют на объём добычи газа. Для этого Василий решил нанять вас как специалиста по построению моделей машинного обучения.

Василий представляет вам набор данных о добыче газа на своих скважинах. Файл с данными вы можете скачать [здесь](https://lms.skillfactory.ru/assets/courseware/v1/11a2ef69ecca8fc5ec1b5c43c8dde935/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/unconv.zip).

Признаки:

* Well — идентификатор скважины;
* Por — пористость скважины (%);
* Perm — проницаемость скважины;
* AI — акустический импеданс ();
* Brittle — коэффициент хрупкости скважины (%);
* TOC — общий органический углерод (%);
* VR — коэффициент отражения витринита (%);
* **Prod** — добыча газа в сутки (млн. кубических футов).

Ваша задача — построить регрессионную модель, которая прогнозирует выработку газа на скважине (целевой признак — **Prod**) на основе остальных характеристик скважины, и проинтерпретировать результаты вашей модели.

In [2]:
data = pd.read_csv('data/unconv.csv')
data.head()

,Well,Por,Perm,AI,Brittle,TOC,VR,Prod
0,1,12.08,2.92,2.80,81.40,1.16,2.31,4165.196191
1,2,12.38,3.53,3.22,46.17,0.89,1.88,3561.146205
2,3,14.02,2.59,4.01,72.80,0.89,2.72,4284.348574
3,4,17.67,6.75,2.63,39.81,1.08,1.88,5098.680869
4,5,17.52,4.57,3.18,10.94,1.51,1.90,3406.132832


Для начала в качестве модели будем использовать простую линейную регрессию.

In [5]:
# Постройте корреляционную матрицу факторов, включив в неё целевой признак. Ответьте на следующие вопросы:

# 1. Выберите топ-3 факторов, наиболее коррелированных с целевой переменной:

A_corr = data.corr()
A_corr

,Well,Por,Perm,AI,Brittle,TOC,VR,Prod
Well,1.000000,0.068927,0.077928,0.041483,-0.079252,0.022624,-0.007279,0.026817
Por,0.068927,1.000000,0.760546,-0.461549,-0.218570,0.711831,0.111860,0.861910
Perm,0.077928,0.760546,1.000000,-0.239636,-0.124017,0.471746,0.051023,0.727426
AI,0.041483,-0.461549,-0.239636,1.000000,0.127599,-0.531864,0.499143,-0.390835
Brittle,-0.079252,-0.218570,-0.124017,0.127599,1.000000,-0.214282,0.317929,0.237155
TOC,0.022624,0.711831,0.471746,-0.531864,-0.214282,1.000000,0.299483,0.654445
VR,-0.007279,0.111860,0.051023,0.499143,0.317929,0.299483,1.000000,0.323182
Prod,0.026817,0.861910,0.727426,-0.390835,0.237155,0.654445,0.323182,1.000000


In [6]:
# 2. Вычислите ранг полученной матрицы корреляций:

np.linalg.matrix_rank(A_corr)

8

In [8]:
# 3. Вычислите определитель матрицы корреляций.
# Ответ округлите до четвёртого знака после точки-разделителя.

np.linalg.det(A_corr).round(4)

0.0007

Создайте матрицу наблюдений. Обозначьте её за X, а вектор правильных ответов — за y.

1. Постройте модель линейной регрессии по методу наименьших квадратов. Для этого используйте матричную формулу NumPy. В качестве ответа укажите полученные оценки коэффициентов модели. Ответ округлите до целого числа.

In [13]:
X

,Well,Por,Perm,AI,Brittle,TOC,VR
0,1,12.08,2.92,2.80,81.40,1.16,2.31
1,2,12.38,3.53,3.22,46.17,0.89,1.88
2,3,14.02,2.59,4.01,72.80,0.89,2.72
3,4,17.67,6.75,2.63,39.81,1.08,1.88
4,5,17.52,4.57,3.18,10.94,1.51,1.90
...,...,...,...,...,...,...,...
195,196,11.95,3.13,2.97,67.18,0.80,2.06
196,197,17.99,9.87,3.38,44.32,0.98,2.08
197,198,12.12,2.27,3.52,57.07,-0.04,1.73
198,199,15.55,4.48,2.48,58.25,1.89,2.35


In [31]:
# составляем матрицу наблюдений и вектор целевой переменной
X = np.column_stack((np.ones(200), data.drop('Prod', axis=1)))
y = data[['Prod']]
# вычисляем OLS-оценку для коэффициентов без стандартизации
w_hat=np.linalg.inv(X.T@X)@X.T@y
print(w_hat.values.round(0))

[[-1232.]
 [    0.]
 [  230.]
 [  116.]
 [ -365.]
 [   25.]
 [  -78.]
 [  785.]]


In [26]:
A_corr

,Well,Por,Perm,AI,Brittle,TOC,VR,Prod
Well,1.000000,0.068927,0.077928,0.041483,-0.079252,0.022624,-0.007279,0.026817
Por,0.068927,1.000000,0.760546,-0.461549,-0.218570,0.711831,0.111860,0.861910
Perm,0.077928,0.760546,1.000000,-0.239636,-0.124017,0.471746,0.051023,0.727426
AI,0.041483,-0.461549,-0.239636,1.000000,0.127599,-0.531864,0.499143,-0.390835
Brittle,-0.079252,-0.218570,-0.124017,0.127599,1.000000,-0.214282,0.317929,0.237155
TOC,0.022624,0.711831,0.471746,-0.531864,-0.214282,1.000000,0.299483,0.654445
VR,-0.007279,0.111860,0.051023,0.499143,0.317929,0.299483,1.000000,0.323182
Prod,0.026817,0.861910,0.727426,-0.390835,0.237155,0.654445,0.323182,1.000000


Далее потренируемся строить предсказание для наблюдений целевой переменной.

1. Постройте прогноз выработки газа для скважины с параметрами, указанными ниже. Чему равна абсолютная ошибка построенного вами прогноза для предложенной скважины (в миллионах кубических футов в день). Ответ округлите до целого числа.

In [23]:
prognosis_values = np.array([1,106,15.32,3.71,3.29,55.99,1.35,2.42])
prediction = prognosis_values@w_hat
abs_error = abs(prediction-4748.315024).round(0)
abs_error

Prod    25.0
dtype: float64

Постройте прогноз выработки газа для всех скважин из обучающего набора данных. Чему равно значение метрики MAPE вашей модели? Ответ приведите в процентах (не указывайте знак процента), округлив его до первого знака после точки-разделителя.

In [25]:
from sklearn import metrics

y_predict = X@w_hat

print('MAPE score: {:.1f} %'.format(metrics.mean_absolute_percentage_error(y, y_predict) * 100))

MAPE score: 3.6 %


Настало время анализа построенной модели. Посмотрите на коэффициенты и сравните их знаки со значениями выборочных корреляций между целевым признаком и факторами, которые вы нашли ранее.


1. Есть ли в вашей модели фактор, при котором коэффициент в модели линейной регрессии противоречит соответствующему коэффициенту корреляции? Например, корреляция говорит, что зависимость между фактором и целью прямая, а модель говорит обратное.

Если такой фактор есть, выберите его название из списка. Если таких факторов несколько, выберите их все:

In [ ]:
# сделано выше (визуально сравнили A_corr и w_hat)

Исключите из данных сильно коррелированные между собой факторы. Под сильной корреляцией в данной задаче будем понимать значения, выше 0.7. Выбирая, какой из коррелированных факторов оставить, руководствуйтесь коэффициентом корреляции с целевой переменной: оставляйте тот фактор, который больше всего коррелирует с объёмом добычи газа.

Также исключите из данных факторы, для которых корреляция с целевой переменной меньше 0.05.

1. Какие факторы вы будете исключать? 

(Well, Perm, TOC)

2. Постройте линейную регрессию на обновлённых после удаления факторов данных по методу наименьших квадратов. Для этого используйте матричную формулу NumPy.

В качестве ответа укажите полученные оценки коэффициентов модели. Ответ округлите до целого числа.

In [37]:
# составляем матрицу наблюдений и вектор целевой переменной
X = np.column_stack((np.ones(200), data.drop(['Prod','Well','Perm','TOC'], axis=1)))
y = data[['Prod']]
# вычисляем OLS-оценку для коэффициентов без стандартизации
w_hat=np.linalg.inv(X.T@X)@X.T@y
print(w_hat.values.round(0))

[[-1835.]
 [  293.]
 [ -200.]
 [   28.]
 [  517.]]


3. Сделайте прогноз для всего обучающего набора данных и рассчитайте метрику MAPE (Mean Absolute Percentage Error). Результат приведите в процентах (не указывайте знак процента), округлив его до первого знака после точки-разделителя.

In [38]:
y_predict = X@w_hat

print('MAPE score: {:.1f} %'.format(metrics.mean_absolute_percentage_error(y, y_predict) * 100))

MAPE score: 4.0 %


In [12]:
# составляем матрицу наблюдений без дополнительного столбца из единиц
X = data.drop(['Prod', axis=1)
y = data['Prod']

# стандартизируем векторы в столбцах матрицы A
X_cent = X - X.mean()
X_st = X_cent/np.linalg.norm(X_cent, axis=0)

# стандартизируем вектор целевой переменной
y_cent = y - y.mean()
y_st = y_cent/np.linalg.norm(y_cent)

# вычислим OLS-оценку для коэффициентов
w_hat_st=np.linalg.inv(X_st.T@X_st)@X_st.T@y_st
print(w_hat_st.values)

[ 0.00295805  0.6893915   0.20282618 -0.20868932  0.35599108 -0.03805996
  0.23812334]
